<a href="https://colab.research.google.com/github/akshatk16/stockPrediction/blob/main/stockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# predict stock prices using ML

# import dependencies
import quandl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [ ]:
# get stock data
quandl.ApiConfig.api_key = "F2p7bTsyUzsth72dKwdX"
df = quandl.get("WIKI/FB")
print(df.head())